# HASHing

In quantum computing the information is encoded using quantum circuits - models that consist of a series of quantum gates that act on qubits. And how is it possible to identify how different two quantum circuits are, especially if the number of qubits and operators is large?  **The answer**: compare their `HASH` values.


![image.jpg](Fig1_hashing.jpg) 
*Fig.1. Schematic representation of three quantum circuits.*


For quantum circuits a “hash” is their unique identifier.  It is generated by a hash function from an input data, where even a small change in the input data results in a significantly different hash value. Comparing hash values, we can verify whether two quantum circuits are equivalent or different, without the need for exhaustive comparisons of their components. A “hash”  is a fixed-size string, such a string might look like this: *35195386635468474207423849615131271890563224714466150433435732026163027928954*. 

In Qiskit there is no built-in hash function for quantum circuits. One can use standard hash functions available in Python (e.g. hashlib library) to generate hash values for quantum circuits. However, basic Python implementation of hashing using built-in id functions causes collisions for specific scenarios* for quantum circuits.
Here we present an example on hash calculation for a quantum circuit using Rivet transpiler function `get_circuit_hash` that solve such problem [1].  

*circuits with not-bound RZ("theta_1") and RZ("theta_2") will have equal hashes.

## 1. Purpose and Relevance

Hashing can be used for:

1. **Circuit Verification and Optimization** [2-5]: as a unique identifier of a quantum circuit the hash allows for easy comparison of quantum circuits and verification of the correctness of a circuit. If two circuits produce the same hash, it indicates that they are equivalent in terms of functionality, even if they are represented differently. This is useful for circuit optimization.

2. **Speed up Computation** [6,7]:  it enables the usage of previously computed circuit results. By storing the hash along with the result, one can quickly retrieve the result of a previously executed circuit if the same circuit is used again.  


3. **Error Detection** [8-10]: Hashing can be used to detect errors or corruption in quantum circuits. If the hash of a computed circuit result does not match the expected hash, it indicates that an error may have occurred during execution or transmission.

4. **Security (quantum communication and cryptography)** [11-14]: Hashing can also be used for security purposes in quantum communication and cryptography. By hashing quantum circuits and comparing hashes, one can ensure the integrity of transmitted quantum information and detect any unauthorized modifications.



## 2. Quantum Circuit Hash Computation Theory


The main steps to compare the hashes of two quantum circuits are as follows:
- **Generate Hash Values**: Compute the hash value of each quantum circuit using a suitable hash function. This function should take into account the structure and operations of the circuits to produce unique hash values for each of them.   
- **Compare Hash Values**.
- **Results Analysis**: test the hash function for a few simple circuits and compare the theoretical predictions with experimental results.

So first, to generate a hash for a circuit the **hash function** is required. A hash function is a mathematical algorithm that takes an input (here, a quantum circuit in the present case) and produces a fixed-size string of characters - the hash.

In `Qiskit` there is no built-in hash function for quantum circuits. One can use standard hash functions available in Python (e.g. `hashlib` library) to generate hash values for quantum circuits. However, basic Python implementation of hashing using built-in id functions causes confusions for specific scenarios for quantum circuits.



The **Rivet Transpiler** package provides a family of functions for the efficient transpilation of quantum circuits. To allow users to easily produce hashes for their circuits. As a solution the function `get_circuit_hash` was built. For more details, please follow the steps below.

## 3. Installation and Imports


To run the code below make sure you have followed and installed Rivet transpiler. Check the installation steps from   **Readme.md**.

#### 3.1. Import Qiskit and Rivet transpiler functions required to run the examples below

To be able execute code below - matplotlib package should be installed

In [1]:
# !pip install matplotlib

In [2]:
###Import qiskit and main settings
import qiskit

from time import time  #will be used to measure execution time
from matplotlib import pyplot as plt  #plot settings

plt.style.use("dark_background") #optional, sets dark_background

###Import Rivet transpiler functions
%cd ../..
from rivet_transpiler import get_circuit_hash
from rivet_transpiler import get_litmus_circuit
from rivet_transpiler import transpile

/mnt/c/Users/mohor/Jupyter/haiqu


#### 3.2. Import Backend
Available backends from IBM Qiskit are listed at [15]. Some backend examples:
- the simplest 5qubit backend (for a simple test start from this one): `FakeBackend5QV2`
- 5qubit backend with noise:  `FakeLimaV2`
- 32-qubit backend: `FakeMontrealV2`

In [3]:
from qiskit_ibm_runtime.fake_provider import FakeLimaV2
backend = FakeLimaV2() 

# qiskit.visualization.plot_gate_map(backend)

![image.png](Fig2_FakeLima_backend.png)

*Fig.2. Schematic representation of FakeLimaV2 backend.*

#### 3.3. Set up `litmus_circuit`
A quantum circuit designed for testing, diagnostic purposes. Litmus circuit** is a minimal quantum circuit which consists of:

- Parametrized RZ gate on each qubit (each parameter name matches the  index of the virtual qubit - to trace consequent permutations of qubits during layout and routing phases of transpilation).

- Circular CNOT gates (necessary to introduce SWAPs during transpilation to not fully connected topology).

In [4]:
QUBITS_COUNT = 3 #define number of qubits
litmus_circuit = get_litmus_circuit(QUBITS_COUNT, "Litmus")
litmus_circuit.draw()

┌──────────────┐          ┌───┐ Litmus 
Litmus_0_0: ┤ Rz(Litmus_0) ├──■───────┤ X ├───░────
            ├──────────────┤┌─┴─┐     └─┬─┘   ░    
Litmus_0_1: ┤ Rz(Litmus_1) ├┤ X ├──■────┼─────░────
            ├──────────────┤└───┘┌─┴─┐  │     ░    
Litmus_0_2: ┤ Rz(Litmus_2) ├─────┤ X ├──■─────░────
            └──────────────┘     └───┘        ░

## 4. Experiments, results

Function `get_circuit_hash` calculates SHA256 hash for a given quantum circuit.

**Hash is based on properties of the circuit gates:**

- Instruction class (RX, CNOT, CRZ, etc.);
- Parameter values (angles);
- Used qubits;
- Used classical bits.

**The following steps should be performed to calculate a circuit hash:**
- Iterate over each gate in the quantum circuit.
- For each gate, extract its properties.
- Convert these properties into a unique string representation.
- Concatenate these string representations for all gates in the circuit.
- Compute the SHA256 hash of the concatenated string.

### 4.1. Compare Simple Hashes

Consider two quantum circuits with the same  gates (`x` and `h`), but they differ by order in which the gates are applied to the qubits. Due to the non-commutative nature of quantum gates, the order in which gates are applied can affect the resulting quantum state. Therefore, the two circuits xh_circuit and hx_circuit may produce different quantum states, even though they consist of the same gates applied to the same qubits.

**Expectations**: for the first example the hash should be the same  (gates are applied to different qubits). For the second example hashes differ because gates act on the same qubit.


#### 4.1.1. Apply gates to different qubits

In [5]:
# Define the circuits
#creates a circuit where X gate (x)  is applied to qubit 0 first and then applies the Hadamard gate (h) to qubit 1
hx_circuit = qiskit.QuantumCircuit(2)
hx_circuit.h(1)
hx_circuit.x(0)

#creates a circuit where Hadamard gate (h)  is applied to qubit 1 first and then applies the X gate (x) to qubit 0
xh_circuit = qiskit.QuantumCircuit(2)
xh_circuit.x(0)
xh_circuit.h(1)

# Visualize the circuits
print("Circuit hx_circuit:")
print(hx_circuit.draw())

print("\nCircuit xh_circuit:")
print(xh_circuit.draw())

# Calculate the hash values
hash_hx_circuit = get_circuit_hash(hx_circuit)
hash_xh_circuit = get_circuit_hash(xh_circuit)

# Compare the hash values
if hash_hx_circuit == hash_xh_circuit:
    print("\nThe circuits have the same hash value.")
else:
    print("\nThe circuits have different hash values.")
# Display Hashes
display(get_circuit_hash(hx_circuit))
display(get_circuit_hash(xh_circuit))

Circuit hx_circuit:
     ┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ H ├
     └───┘

Circuit xh_circuit:
     ┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ H ├
     └───┘

The circuits have the same hash value.


74331646843496876009284120309449186196481076817318235757987417819627350351433

74331646843496876009284120309449186196481076817318235757987417819627350351433

#### 4.1.2. Apply gates to the same qubit

In [12]:
import hashlib

from collections import deque

def get_circuit_hash(circuit, decomposition_level=None):

    hash_object = hashlib.sha256(b'')

    # Circuit Traversal

    initial_level = 0
    initial_qubit_base = list(range(circuit.num_qubits))
    initial_bit_base = list(range(circuit.num_clbits))

    initial_record = (initial_level,
                      initial_qubit_base,
                      initial_bit_base,
                      circuit)

    queue = deque([initial_record])

    while queue:

        current_record = queue.popleft()

        level, qubit_base, bit_base, current_circuit = current_record

        current_dag = qiskit.converters.circuit_to_dag(current_circuit)

        for node in current_dag.topological_op_nodes():

            # Relative Indices

            relative_qubits = [current_circuit.find_bit(qubit).index
                               for qubit in node.qargs]

            relative_bits = [current_circuit.find_bit(bit).index
                             for bit in node.cargs]

            # Absolute Indices

            absolute_qubits = [qubit_base[qubit] for qubit in relative_qubits]
            absolute_bits = [bit_base[bit] for bit in relative_bits]

            # Sub Circuit

            operation = node.op

            sub_circuit = operation.definition

            if level == decomposition_level or sub_circuit is None:

                # Calculate Hash of Leaf Node

                # Collect Values

                values = [absolute_qubits,
                          absolute_bits,
                          operation.name]

                # Collect Parameters

                for parameter in operation.params:

                    if isinstance(parameter, qiskit.circuit.parameter.ParameterExpression):

                        parameter = None

                    values.append(parameter)

                # Update Hash

                for value in values:

                    encoded_value = repr(value).encode('utf-8')

                    hash_object.update(encoded_value)

            else:

                # Add new Record to the Queue

                new_record = (level + 1,
                              absolute_qubits, absolute_bits,
                              sub_circuit)

                queue.append(new_record)

    # Digest Hash

    hash_bytes = hash_object.digest()

    hash_value = int.from_bytes(hash_bytes, byteorder='little')

    return hash_value

In [13]:
# Define the circuits
#creates a circuit where the Hadamard gate (h) and then X gate (x)  are applied to qubit 1
hx_circuit = qiskit.QuantumCircuit(2)
hx_circuit.h(1)
hx_circuit.x(1)

#creates a circuit where X gate (x)  and then the Hadamard gate (h) are applied to qubit 1
xh_circuit = qiskit.QuantumCircuit(2)
xh_circuit.x(1)
xh_circuit.h(1)

# Visualize the circuits
print("Circuit hx_circuit:")
print(hx_circuit.draw())

print("\nCircuit xh_circuit:")
print(xh_circuit.draw())

# Calculate the hash values
hash_hx_circuit = get_circuit_hash(hx_circuit)
hash_xh_circuit = get_circuit_hash(xh_circuit)

# Compare the hash values
if hash_hx_circuit == hash_xh_circuit:
    print("\nThe circuits have the same hash value.")
else:
    print("\nThe circuits have different hash values.")
# Display Hashes
display(get_circuit_hash(hx_circuit))
display(get_circuit_hash(xh_circuit))

Circuit hx_circuit:
               
q_0: ──────────
     ┌───┐┌───┐
q_1: ┤ H ├┤ X ├
     └───┘└───┘

Circuit xh_circuit:
               
q_0: ──────────
     ┌───┐┌───┐
q_1: ┤ X ├┤ H ├
     └───┘└───┘

The circuits have the same hash value.


8895418907420795653914132157143967161278273931489570184451554769513788291599

8895418907420795653914132157143967161278273931489570184451554769513788291599

### 4.2. Compare Parameter Hashes
Bind the parameters of a quantum circuit to specific values, adjusting them with an optional `offset` to prepare the circuits with specific parameter values. We define two circuits created by binding the parameters of `litmus_circuit` with the same `offset`, and the last circuit - with different `offset`.

**Expectations**: the first two circuits have the same hashes, and for the third circuit it differs.

In [6]:
def bind_parameters_with_offset(circuit, offset=0):

    bound_circuit = circuit.copy()

    for index, parameter in enumerate(bound_circuit.parameters):

        bound_circuit.assign_parameters(
            {parameter: index + offset},
            inplace=True)

    return bound_circuit

In [7]:
# Define Bound Circuits
bound_circuit = bind_parameters_with_offset(litmus_circuit, offset=0)
bound_circuit_same_parameters = bind_parameters_with_offset(litmus_circuit, offset=0)
bound_circuit_other_parameters = bind_parameters_with_offset(litmus_circuit, offset=1)

In [8]:
# Display Circuits
display(bound_circuit.draw(fold=-1))
display(bound_circuit_same_parameters.draw(fold=-1))
display(bound_circuit_other_parameters.draw(fold=-1))

┌───────┐          ┌───┐ Litmus 
Litmus_0_0: ┤ Rz(0) ├──■───────┤ X ├───░────
            ├───────┤┌─┴─┐     └─┬─┘   ░    
Litmus_0_1: ┤ Rz(1) ├┤ X ├──■────┼─────░────
            ├───────┤└───┘┌─┴─┐  │     ░    
Litmus_0_2: ┤ Rz(2) ├─────┤ X ├──■─────░────
            └───────┘     └───┘        ░

┌───────┐          ┌───┐ Litmus 
Litmus_0_0: ┤ Rz(0) ├──■───────┤ X ├───░────
            ├───────┤┌─┴─┐     └─┬─┘   ░    
Litmus_0_1: ┤ Rz(1) ├┤ X ├──■────┼─────░────
            ├───────┤└───┘┌─┴─┐  │     ░    
Litmus_0_2: ┤ Rz(2) ├─────┤ X ├──■─────░────
            └───────┘     └───┘        ░

┌───────┐          ┌───┐ Litmus 
Litmus_0_0: ┤ Rz(1) ├──■───────┤ X ├───░────
            ├───────┤┌─┴─┐     └─┬─┘   ░    
Litmus_0_1: ┤ Rz(2) ├┤ X ├──■────┼─────░────
            ├───────┤└───┘┌─┴─┐  │     ░    
Litmus_0_2: ┤ Rz(3) ├─────┤ X ├──■─────░────
            └───────┘     └───┘        ░

In [9]:
# Display Hashes
display(get_circuit_hash(bound_circuit))
display(get_circuit_hash(bound_circuit_same_parameters))
display(get_circuit_hash(bound_circuit_other_parameters))

10033460680529423832288923830260624451335426098717749005707346276546548518421

10033460680529423832288923830260624451335426098717749005707346276546548518421

96311980127339149651295543283381731758114225806793066312244782303502616987800


### 4.3. Compare Structure Hashes

Define three circuits with a given `seed` values provided to the transpiler.
The `seed` values influences the optimization and decomposition processes performed by the transpiler, so  potentially the gate sequences and structural characteristics of the transpiled circuits may differ. First two circuits  are transpiled using the same `seed` value, while the 3rd circuit is transpiled using a different.

**Expectations**: first two circuits should have the same hashes, and the for the third it differs.


In [10]:
# Define 3 circuits
#for each circuit different properties can be added. For example:
    # optimization_level=3,
    # initial_layout=[1, 2, 3]

transpiled_litmus_circuit = transpile(
    litmus_circuit,
    backend,
    seed_transpiler=1234,
)

transpiled_litmus_circuit_same_seed = transpile(
    litmus_circuit,
    backend,
    seed_transpiler=1234,
)

transpiled_litmus_circuit_other_seed = transpile(
    litmus_circuit,
    backend,
    seed_transpiler=777,
)

In [11]:
# Display Circuits

display(transpiled_litmus_circuit.draw(fold=-1))
display(transpiled_litmus_circuit_same_seed.draw(fold=-1))
display(transpiled_litmus_circuit_other_seed.draw(fold=-1))

┌──────────────┐          ┌───┐          ┌───┐ Litmus 
Litmus_0_2 -> 0 ┤ Rz(Litmus_2) ├───────■──┤ X ├──■───────┤ X ├───░────
                ├──────────────┤     ┌─┴─┐└─┬─┘┌─┴─┐┌───┐└─┬─┘   ░    
Litmus_0_0 -> 1 ┤ Rz(Litmus_0) ├──■──┤ X ├──■──┤ X ├┤ X ├──■─────░────
                ├──────────────┤┌─┴─┐└───┘     └───┘└─┬─┘        ░    
Litmus_0_1 -> 2 ┤ Rz(Litmus_1) ├┤ X ├─────────────────■──────────░────
                └──────────────┘└───┘                            ░    
 ancilla_0 -> 3 ──────────────────────────────────────────────────────
                                                                      
 ancilla_1 -> 4 ──────────────────────────────────────────────────────

┌──────────────┐          ┌───┐          ┌───┐ Litmus 
Litmus_0_2 -> 0 ┤ Rz(Litmus_2) ├───────■──┤ X ├──■───────┤ X ├───░────
                ├──────────────┤     ┌─┴─┐└─┬─┘┌─┴─┐┌───┐└─┬─┘   ░    
Litmus_0_0 -> 1 ┤ Rz(Litmus_0) ├──■──┤ X ├──■──┤ X ├┤ X ├──■─────░────
                ├──────────────┤┌─┴─┐└───┘     └───┘└─┬─┘        ░    
Litmus_0_1 -> 2 ┤ Rz(Litmus_1) ├┤ X ├─────────────────■──────────░────
                └──────────────┘└───┘                            ░    
 ancilla_0 -> 3 ──────────────────────────────────────────────────────
                                                                      
 ancilla_1 -> 4 ──────────────────────────────────────────────────────

┌──────────────┐     ┌───┐                       ░    
Litmus_0_2 -> 0 ┤ Rz(Litmus_2) ├─────┤ X ├─────────────────■─────░────
                └──────────────┘     └─┬─┘                 │     ░    
                ┌──────────────┐┌───┐  │  ┌───┐     ┌───┐┌─┴─┐ Litmus 
Litmus_0_1 -> 1 ┤ Rz(Litmus_1) ├┤ X ├──■──┤ X ├──■──┤ X ├┤ X ├───░────
                ├──────────────┤└─┬─┘     └─┬─┘┌─┴─┐└─┬─┘└───┘   ░    
Litmus_0_0 -> 2 ┤ Rz(Litmus_0) ├──■─────────■──┤ X ├──■──────────░────
                └──────────────┘               └───┘             ░    
 ancilla_0 -> 3 ──────────────────────────────────────────────────────
                                                                      
 ancilla_1 -> 4 ──────────────────────────────────────────────────────

In [12]:
# Display Hashes

display(get_circuit_hash(transpiled_litmus_circuit))
display(get_circuit_hash(transpiled_litmus_circuit_same_seed))
display(get_circuit_hash(transpiled_litmus_circuit_other_seed))

95057451833288437742216986931635592134430082860549167753734612993304038033677

95057451833288437742216986931635592134430082860549167753734612993304038033677

22263785752630235322650324771950494147518387134149664363329640026482765087331

### 4.4. Hashing Time
Generate quantum circuits with a varying number of layers.
It iterates over a range of layer counts, constructs the circuits, decomposes them into elementary gates, calculates the number of gates, computes the hash of the decomposed circuits, and measures the elapsed time for each iteration.

The results, including the gate counts, layer counts, and elapsed times, are stored in lists for further visualization.

In [13]:
QUBITS_COUNT = 100
MAX_LAYERS_COUNT = 5

GATES = ['x', 'y', 'z',
         'rx', 'ry', 'rz',
         'rxx', 'ryy', 'rzz',
         'swap', 'i']

gates_counts = []
layers_counts = []
elapsed_times = []

for layers_count in range(MAX_LAYERS_COUNT):

    circuit = qiskit.circuit.library.EfficientSU2(QUBITS_COUNT,
                                                  reps=layers_count,
                                                  su2_gates=GATES,
                                                  entanglement="circular",
                                                  skip_final_rotation_layer=True)


    circuit_decomposed = circuit.decompose(None, 1)
    gates_count = len(circuit_decomposed.data)
    print("gates_count:", gates_count)

    start_time = time()
    circuit_hash = get_circuit_hash(circuit_decomposed)
    elapsed_time = time() - start_time
    gates_counts.append(gates_count)
    layers_counts.append(layers_count)
    elapsed_times.append(elapsed_time)
    print(f"elapsed_time:{elapsed_time:.5f}")

    # circuit_decomposed.draw(fold=-1)

gates_count: 0
elapsed_time:0.00088
gates_count: 1000
elapsed_time:0.01037
gates_count: 2000
elapsed_time:0.01810
gates_count: 3000
elapsed_time:0.02853
gates_count: 4000
elapsed_time:0.03472


From the plot  for the hashing time as a function of gate counts **a linear dependency is observed**:

In [14]:
#Plot Hashing Time as a function of the number of gates

# plt.title("Hashing Time")

# plt.xlabel("Gates count")
# plt.ylabel("Time, seconds")

# plt.plot(gates_counts, elapsed_times);

![image.png](Fig3_timehashing.png) 

*Fig.3. Hashing time as a function of gate counts for the circuit above.*

### 4.5. Benchmarking

`timeit -r 10 -n 100`: measure the average execution time of the statement (10 runs * 100 iterations)

`get_circuit_hash(circuit)`: present the given circuit hash

`timeit -r 1 -n 1`:   measure the execution time of a single run of the statement

`get_circuit_hash(circuit_decomposed)`: calculates a hash value for a decomposed circuit

## 5. Conclusions
Our exploration also demonstrates the importance and relevance of proper hashing in quantum Qiskit circuits, exemplified by the observation that different circuits produce distinct hashes, facilitating circuit verification and optimization. Experiments comparing simple hashes, parameter hashes, structure hashes, and hashing time (scales linear as a function of gate counts) can provide further insights into the efficacy and performance of hashing techniques in quantum computing.


## References

[1] Inspired by Qiskit `soft_compare` gate function:
https://github.com/Qiskit/qiskit/blob/main/qiskit/circuit/instruction.py#L227

[2] https://link.springer.com/referenceworkentry/10.1007/978-981-15-6401-7_43-1

[3] https://www.cs.cmu.edu/~zhihaoj2/papers/quartz-pldi22.pdf

[4] https://eprint.iacr.org/2020/1273.pdf

[5] https://arxiv.org/pdf/2111.11387.pdf

[6] https://www.linkedin.com/pulse/quantum-acceleration-fraud-real-lets-try-break-hash-function-parisel/

[7] https://quantumcomputing.stackexchange.com/questions/25697/whats-the-effective-speed-of-quantum-computers-circa-2022

[8] https://ceur-ws.org/Vol-3513/paper06.pdf

[9] https://security.stackexchange.com/questions/198027/error-detection-capability-of-sha-256

[10] https://www.sciencedirect.com/science/article/abs/pii/S0026271413003156

[11] https://learning.quantum.ibm.com/course/practical-introduction-to-quantum-safe-cryptography/cryptographic-hash-functions

[12] https://en.wikipedia.org/wiki/Cryptographic_hash_function

[13] https://onlinelibrary.wiley.com/doi/abs/10.1002/ett.4460050406   https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=f3899323a7b6d060ce720e6cb6e97c390164ef63

[14] https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=85abc4805adb741b0f8c962794d2ab4dac975c5f

[15] Qiskit fake backends: https://docs.quantum.ibm.com/api/qiskit/0.37/providers_fake_provider